##Урок 8. Рекуррентные нейронные сети RNN LSTM GRU
Данные берем отызывы за лето (прошлый урок)

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи

построить свёрточные архитектуры

построить различные архитектуры с RNN

построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [1]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 549.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 49.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=b51f0de3b4d6235615911dd4e26ad55538f55e49656aca730c5de7a2fed3da00
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
import numpy as np
import pandas as pd

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from pymorphy2 import MorphAnalyzer

import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, Flatten, SimpleRNN, LSTM, GRU, Reshape
from keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_excel('/content/drive/MyDrive/nlp/les7/отзывы за лето.xls')
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [5]:
max_words = 20000
max_len = 150
num_classes = 5

epochs = 20
batch_size = 512
print_batch_n = 100

In [6]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
stopwordslist = stopwords.words("russian")
ptrn = r'[^a-zA-Zа-яА-Я0-9]'

morpher = MorphAnalyzer()

def words_only(text):
    text=str(text)
    return text.lower()

def remove_punkt(text):
    return re.sub(ptrn, ' ', text)

def to_token(text):
    return nltk.tokenize.word_tokenize(text)

def remove_stopwords(text):
    text_list = [w for w in text if w not in stopwordslist]
    return ' '.join(word for word in text_list)

def morphe_text(text):
    text = [morpher.parse(word)[0].normal_form for word in text.split() if word not in stopwordslist]
    return " ".join(text)

def normalize_text(text):
    text = words_only(text)
    text = remove_punkt(text)
    text = to_token(text)
    text = remove_stopwords(text)
    text = morphe_text(text)
    return text

In [8]:
data['normalized_content'] = data['Content'].apply(normalize_text)
data.head()

,Rating,Content,Date,normalized_content
0,5,It just works!,2017-08-14,it just works
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложение минус хотеть слишко...
2,5,Отлично все,2017-08-14,отлично
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать 1 работа антивирус далёкий нику...
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро


In [9]:
train_corpus = " ".join(data['normalized_content'])
train_tokens = word_tokenize(train_corpus)
train_tokens_filtered = [word for word in train_tokens if word.isalnum()]

In [10]:
dist = FreqDist(train_tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
voc = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [11]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in voc:
            result.append(voc[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

data_train = np.asarray(
    [text_to_sequence(text, max_len) for text in data['normalized_content']],
    dtype=np.int32)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(data_train, data.Rating, test_size=0.3, random_state=1)

le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

###CNN

In [13]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(GlobalMaxPool1D())
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 44s 2s/step - loss: 1.1404 - accuracy: 0.7029 - val_loss: 1.0905 - val_accuracy: 0.7155
Epoch 2/5
23/23 [==============================] - 35s 2s/step - loss: 0.8729 - accuracy: 0.7027 - val_loss: 0.8508 - val_accuracy: 0.7128
Epoch 3/5
23/23 [==============================] - 34s 1s/step - loss: 0.7301 - accuracy: 0.7073 - val_loss: 0.7849 - val_accuracy: 0.7107
Epoch 4/5
23/23 [==============================] - 38s 2s/step - loss: 0.6764 - accuracy: 0.7388 - val_loss: 0.7612 - val_accuracy: 0.7459
Epoch 5/5
23/23 [==============================] - 34s 1s/step - loss: 0.6387 - accuracy: 0.7639 - val_loss: 0.7542 - val_accuracy: 0.7356


In [14]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = pd.DataFrame(columns=['model', 'Test score', 'Test accuracy'])

metrics_df = metrics_df.append({
    'model': 'CNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 4s 264ms/step - loss: 0.7808 - accuracy: 0.7139
Test score: 0.7808461785316467
Test accuracy: 0.7139399647712708


<ipython-input-14-f8505f291b9a>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###SimpleRNN

In [15]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 21s 766ms/step - loss: 1.3976 - accuracy: 0.6324 - val_loss: 1.0130 - val_accuracy: 0.7186
Epoch 2/5
23/23 [==============================] - 17s 754ms/step - loss: 0.8280 - accuracy: 0.7347 - val_loss: 0.7657 - val_accuracy: 0.7349
Epoch 3/5
23/23 [==============================] - 17s 749ms/step - loss: 0.6886 - accuracy: 0.7563 - val_loss: 0.6858 - val_accuracy: 0.7701
Epoch 4/5
23/23 [==============================] - 19s 796ms/step - loss: 0.6247 - accuracy: 0.7792 - val_loss: 0.6726 - val_accuracy: 0.7784
Epoch 5/5
23/23 [==============================] - 16s 723ms/step - loss: 0.5767 - accuracy: 0.7924 - val_loss: 0.6689 - val_accuracy: 0.7795


In [16]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'SimpleRNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 3s 226ms/step - loss: 0.6846 - accuracy: 0.7656
Test score: 0.6845921277999878
Test accuracy: 0.7655695676803589


<ipython-input-16-726e1a524894>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###LSTM

In [17]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 59s 2s/step - loss: 1.5357 - accuracy: 0.6904 - val_loss: 1.3709 - val_accuracy: 0.7155
Epoch 2/5
23/23 [==============================] - 49s 2s/step - loss: 1.1005 - accuracy: 0.7030 - val_loss: 0.8914 - val_accuracy: 0.7155
Epoch 3/5
23/23 [==============================] - 49s 2s/step - loss: 0.8019 - accuracy: 0.7030 - val_loss: 0.7636 - val_accuracy: 0.7155
Epoch 4/5
23/23 [==============================] - 49s 2s/step - loss: 0.7166 - accuracy: 0.7033 - val_loss: 0.7362 - val_accuracy: 0.7155
Epoch 5/5
23/23 [==============================] - 49s 2s/step - loss: 0.6728 - accuracy: 0.7130 - val_loss: 0.7226 - val_accuracy: 0.7235


In [18]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'LSTM',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 9s 727ms/step - loss: 0.7467 - accuracy: 0.7131
Test score: 0.7466565370559692
Test accuracy: 0.7131332755088806


<ipython-input-18-c7475f127a67>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###GRU

In [19]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 47s 2s/step - loss: 1.5207 - accuracy: 0.6782 - val_loss: 1.2948 - val_accuracy: 0.7155
Epoch 2/5
23/23 [==============================] - 39s 2s/step - loss: 1.0127 - accuracy: 0.7030 - val_loss: 0.8323 - val_accuracy: 0.7155
Epoch 3/5
23/23 [==============================] - 37s 2s/step - loss: 0.7791 - accuracy: 0.7030 - val_loss: 0.7465 - val_accuracy: 0.7155
Epoch 4/5
23/23 [==============================] - 41s 2s/step - loss: 0.6694 - accuracy: 0.7666 - val_loss: 0.6936 - val_accuracy: 0.7809
Epoch 5/5
23/23 [==============================] - 41s 2s/step - loss: 0.6187 - accuracy: 0.7870 - val_loss: 0.6843 - val_accuracy: 0.7805


In [20]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'GRU',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 6s 450ms/step - loss: 0.7113 - accuracy: 0.7601
Test score: 0.711312472820282
Test accuracy: 0.7600839138031006


<ipython-input-20-c7a60f594fef>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


# ###CNN -> LSTM

In [21]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 65s 3s/step - loss: 1.5270 - accuracy: 0.2719 - val_loss: 1.3688 - val_accuracy: 0.7155
Epoch 2/5
23/23 [==============================] - 60s 3s/step - loss: 1.2837 - accuracy: 0.6210 - val_loss: 1.1231 - val_accuracy: 0.7155
Epoch 3/5
23/23 [==============================] - 63s 3s/step - loss: 1.0299 - accuracy: 0.7031 - val_loss: 0.8828 - val_accuracy: 0.7155
Epoch 4/5
23/23 [==============================] - 60s 3s/step - loss: 0.8068 - accuracy: 0.7274 - val_loss: 0.7157 - val_accuracy: 0.7743
Epoch 5/5
23/23 [==============================] - 60s 3s/step - loss: 0.6572 - accuracy: 0.7827 - val_loss: 0.6671 - val_accuracy: 0.7767


In [22]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'CNN -> LSTM',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 9s 658ms/step - loss: 0.6974 - accuracy: 0.7641
Test score: 0.6973655223846436
Test accuracy: 0.7641174793243408


<ipython-input-22-9fb914ee426c>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###CNN -> GRU

In [23]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 57s 2s/step - loss: 1.1243 - accuracy: 0.6684 - val_loss: 0.8117 - val_accuracy: 0.7155
Epoch 2/5
23/23 [==============================] - 50s 2s/step - loss: 0.7844 - accuracy: 0.7030 - val_loss: 0.7198 - val_accuracy: 0.7155
Epoch 3/5
23/23 [==============================] - 51s 2s/step - loss: 0.7065 - accuracy: 0.7186 - val_loss: 0.6901 - val_accuracy: 0.7494
Epoch 4/5
23/23 [==============================] - 49s 2s/step - loss: 0.6458 - accuracy: 0.7644 - val_loss: 0.6786 - val_accuracy: 0.7681
Epoch 5/5
23/23 [==============================] - 52s 2s/step - loss: 0.5920 - accuracy: 0.7930 - val_loss: 0.6826 - val_accuracy: 0.7701


In [24]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'CNN -> GRU',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 9s 679ms/step - loss: 0.7145 - accuracy: 0.7510
Test score: 0.7145295739173889
Test accuracy: 0.751048743724823


<ipython-input-24-3cab87a72cc7>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###CNN -> SimpleRNN

In [25]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 33s 1s/step - loss: 1.2067 - accuracy: 0.6385 - val_loss: 0.9407 - val_accuracy: 0.7155
Epoch 2/5
23/23 [==============================] - 30s 1s/step - loss: 0.8779 - accuracy: 0.7190 - val_loss: 0.7898 - val_accuracy: 0.7435
Epoch 3/5
23/23 [==============================] - 31s 1s/step - loss: 0.7516 - accuracy: 0.7487 - val_loss: 0.7103 - val_accuracy: 0.7594
Epoch 4/5
23/23 [==============================] - 30s 1s/step - loss: 0.6680 - accuracy: 0.7678 - val_loss: 0.6732 - val_accuracy: 0.7698
Epoch 5/5
23/23 [==============================] - 31s 1s/step - loss: 0.6103 - accuracy: 0.7848 - val_loss: 0.6706 - val_accuracy: 0.7691


In [26]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'CNN -> SimpleRNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 5s 419ms/step - loss: 0.7016 - accuracy: 0.7472
Test score: 0.7016360759735107
Test accuracy: 0.7471765279769897


<ipython-input-26-13369e4be48f>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###SimpleRNN -> CNN

In [27]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 43s 2s/step - loss: 1.2453 - accuracy: 0.6784 - val_loss: 0.8958 - val_accuracy: 0.7235
Epoch 2/5
23/23 [==============================] - 41s 2s/step - loss: 0.8566 - accuracy: 0.7210 - val_loss: 0.7641 - val_accuracy: 0.7401
Epoch 3/5
23/23 [==============================] - 40s 2s/step - loss: 0.7205 - accuracy: 0.7511 - val_loss: 0.6973 - val_accuracy: 0.7625
Epoch 4/5
23/23 [==============================] - 41s 2s/step - loss: 0.6418 - accuracy: 0.7705 - val_loss: 0.6803 - val_accuracy: 0.7688
Epoch 5/5
23/23 [==============================] - 43s 2s/step - loss: 0.5920 - accuracy: 0.7819 - val_loss: 0.6931 - val_accuracy: 0.7705


In [28]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'SimpleRNN -> CNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 7s 529ms/step - loss: 0.7135 - accuracy: 0.7540
Test score: 0.713502824306488
Test accuracy: 0.7539528608322144


<ipython-input-28-2d413a51352c>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###LSTM -> CNN

In [29]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(LSTM(128, return_sequences=True))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 81s 3s/step - loss: 1.3079 - accuracy: 0.6776 - val_loss: 1.0080 - val_accuracy: 0.7155
Epoch 2/5
23/23 [==============================] - 71s 3s/step - loss: 0.9236 - accuracy: 0.7207 - val_loss: 0.7983 - val_accuracy: 0.7477
Epoch 3/5
23/23 [==============================] - 71s 3s/step - loss: 0.7279 - accuracy: 0.7609 - val_loss: 0.7083 - val_accuracy: 0.7677
Epoch 4/5
23/23 [==============================] - 75s 3s/step - loss: 0.6137 - accuracy: 0.7879 - val_loss: 0.6665 - val_accuracy: 0.7746
Epoch 5/5
23/23 [==============================] - 70s 3s/step - loss: 0.5391 - accuracy: 0.8045 - val_loss: 0.6868 - val_accuracy: 0.7729


In [30]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'LSTM -> CNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 11s 875ms/step - loss: 0.6942 - accuracy: 0.7636
Test score: 0.6941651701927185
Test accuracy: 0.7636334300041199


<ipython-input-30-99076512101e>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###GRU -> CNN

In [31]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(GRU(128, return_sequences=True))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 72s 3s/step - loss: 1.2132 - accuracy: 0.6865 - val_loss: 0.9262 - val_accuracy: 0.7204
Epoch 2/5
23/23 [==============================] - 61s 3s/step - loss: 0.8383 - accuracy: 0.7240 - val_loss: 0.7229 - val_accuracy: 0.7535
Epoch 3/5
23/23 [==============================] - 66s 3s/step - loss: 0.6732 - accuracy: 0.7618 - val_loss: 0.6655 - val_accuracy: 0.7688
Epoch 4/5
23/23 [==============================] - 61s 3s/step - loss: 0.5993 - accuracy: 0.7818 - val_loss: 0.6835 - val_accuracy: 0.7694
Epoch 5/5
23/23 [==============================] - 60s 3s/step - loss: 0.5497 - accuracy: 0.8003 - val_loss: 0.6877 - val_accuracy: 0.7663


In [32]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'GRU -> CNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 9s 661ms/step - loss: 0.7006 - accuracy: 0.7551
Test score: 0.7006334662437439
Test accuracy: 0.7550823092460632


<ipython-input-32-1cb997e7e0ac>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###Итог

In [33]:
metrics_df.sort_values(by='Test score', ascending=False)

,model,Test score,Test accuracy
0,CNN,0.780846,0.713940
2,LSTM,0.746657,0.713133
5,CNN -> GRU,0.714530,0.751049
7,SimpleRNN -> CNN,0.713503,0.753953
3,GRU,0.711312,0.760084
6,CNN -> SimpleRNN,0.701636,0.747177
9,GRU -> CNN,0.700633,0.755082
4,CNN -> LSTM,0.697366,0.764117
8,LSTM -> CNN,0.694165,0.763633
1,SimpleRNN,0.684592,0.765570


In [34]:
metrics_df.sort_values(by='Test accuracy', ascending=False)

,model,Test score,Test accuracy
1,SimpleRNN,0.684592,0.765570
4,CNN -> LSTM,0.697366,0.764117
8,LSTM -> CNN,0.694165,0.763633
3,GRU,0.711312,0.760084
9,GRU -> CNN,0.700633,0.755082
7,SimpleRNN -> CNN,0.713503,0.753953
5,CNN -> GRU,0.714530,0.751049
6,CNN -> SimpleRNN,0.701636,0.747177
0,CNN,0.780846,0.713940
2,LSTM,0.746657,0.713133
